In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
import numpy as np
import pandas as pd

In [4]:
from drillbit import BitcoinEnvironmentUtility
from drillbit.__new_objects import Rig as RigManager, Product as ProductManager, \
    Cooling as CoolingManager, Project as ProjectManager, \
    OverClock, RigOperator
from drillbit.__new_units__ import AbstractBaseUnit, MagnitudeTable, Hashes, HashRate, Time, Power, Energy, Efficiency, EnergyPrice


from environment.serializers import EnvironmentSerializer as EnviroSer, BlockScheduleSerializer as BSchedSer, \
    BitcoinPriceSerializer as BPSer, TransactionFeesSerializer as FeeSer, HashRateSerializer as HRSer
from environment.models import Environment, BlockSchedule, BitcoinPrice, TransactionFees

from products.models import Rig, Cooling, HeatRejection, Electrical, RejectionCurve
from products.serializers import RigSerializer, CoolingSerializer, HeatRejectionSerializer, ElectricalSerializer

from projects.models import RigForProject, InfraForProject, Project, Projects
from projects.serializers import RigForProjectSerializer, InfraForProjectSerializer, ProjectSerializer, ProjectsSerializer, \
    ProjectScalingSerializer, ProjectCostsSerializer

In [5]:
import numpy as np
import pandas as pd
import numpy_financial as npf

import finstat as fs

from drillbit.statements.funcs import total_energy, win_percentage, \
    expected_difficulty, hashes_to_hash_rate, win_percentage


In [6]:
from drillbit.statements.statements import init_environment, ProjectTemplate

In [7]:
rej = HeatRejection.objects.latest('created_at')

In [8]:
btc = BitcoinEnvironmentUtility()

env = Environment.objects.latest('created_at')
blocks = BSchedSer(env.block_schedule, bitcoin_utility=btc)
price = BPSer(env.bitcoin_price, bitcoin_utility=btc)
fees = FeeSer(env.transaction_fees, bitcoin_utility=btc)
hash_rate = HRSer(env.hash_rate, bitcoin_utility=btc)

In [9]:
env = init_environment(
    blocks.to_schedule(),
    price.to_schedule().forecast,
    fees.to_schedule().forecast,
    hash_rate.to_schedule().forecast * 1e6 * 1e12 # convert from M TH/s to H/s
)

In [10]:
project = Project.objects.latest('created_at')
project = project.as_drillbit_object()

In [11]:
stat = ProjectTemplate(env, project)

In [12]:
stat.istat

period,2023-02-16 12:40,2023-02-16 12:50,2023-02-16 13:00,2023-02-16 13:10,2023-02-16 13:20,2023-02-16 13:30,2023-02-16 13:40,2023-02-16 13:50,2023-02-16 14:00,2023-02-16 14:10,...,2029-07-21 16:20,2029-07-21 16:30,2029-07-21 16:40,2029-07-21 16:50,2029-07-21 17:00,2029-07-21 17:10,2029-07-21 17:20,2029-07-21 17:30,2029-07-21 17:40,2029-07-21 17:50
Item,,,,,,,,,,,,,,,,,,,,,
Revenue - Reward,10.870092,10.870756,10.870482,10.875377,10.873249,10.874923,10.874661,10.876737,10.873541,10.871969,...,7.160327,7.157232,7.157817,7.15715,7.15604,7.154407,7.151341,7.148347,7.149208,7.149501
Revenue - Fees,34771.271215,34762.058609,34756.815757,34769.588344,34766.724628,34777.476452,34776.654276,34785.694285,34777.552133,34766.947871,...,67943064.427145,67910569.306326,67911443.441252,67921315.919055,67914601.969479,67887105.880769,67872925.391863,67855421.179603,67855352.957671,67871669.663557
Gross Revenue,34782.141306,34772.929366,34767.68624,34780.463722,34777.597877,34788.351375,34787.528937,34796.571021,34788.425674,34777.819841,...,67943071.587472,67910576.463558,67911450.599069,67921323.076205,67914609.125519,67887113.035175,67872932.543204,67855428.32795,67855360.106879,67871676.813059
Pool Fees,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Net Revenue,34782.141306,34772.929366,34767.68624,34780.463722,34777.597877,34788.351375,34787.528937,34796.571021,34788.425674,34777.819841,...,67943071.587472,67910576.463558,67911450.599069,67921323.076205,67914609.125519,67887113.035175,67872932.543204,67855428.32795,67855360.106879,67871676.813059
Energy Expenses,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,...,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303,303.030303
Gross Profit,-292.160211,-292.159547,-292.159821,-292.154926,-292.157054,-292.15538,-292.155642,-292.153566,-292.156762,-292.158334,...,-295.869976,-295.873071,-295.872486,-295.873153,-295.874263,-295.875896,-295.878962,-295.881956,-295.881095,-295.880802
Operations,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Property Tax,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
infra = project.infrastructure[0]

In [ ]:
amortize(infra.price, infra.quantity, infra.amortization, periods)

In [ ]:
amortize(
    mine.cost_of_cooling, 
    1, 
    mine.cooling.amort, 
    mine.implement.start_in_blocks(), 
    periods=stat.periods
)


In [ ]:
stat.istat.add_account(
    amortize(
        mine.cost_of_cooling, 
        1, 
        mine.cooling.amort, 
        mine.implement.start_in_blocks(), 
        periods=stat.periods
    ), name='Cooling Amortization', short_name='cool_amort'
)


In [ ]:
block_schedule = blocks.to_schedule()
n = block_schedule.shape[0]

In [ ]:
periods = block_schedule.period
number_of_rigs = np.repeat(project.rigs.schedule, n)

In [ ]:
stat.env

In [ ]:
project.rigs.schedule

In [ ]:
energy_price = np.repeat(project.energy_price, n)
opex = np.repeat(0, n)
property_taxes = np.repeat(0, n)
pue = project.project_pue


In [ ]:
project.rigs.schedule

In [ ]:
power_per_rig = project.rigs.OC.power_by_factor().consumption_per_block()
power_per_rig_in_joules = project.rigs.OC.power_by_factor().consumption_per_block().in_joules()

In [ ]:
hash_rate = project.rigs.OC.hash_rate_by_factor().raw

In [ ]:
hashes = project.rigs.OC.hash_rate_by_factor().hashes_per_block().raw

In [ ]:
hashes

In [ ]:
pass in:

project object
rig objects
infrastructure objects

In [ ]:
def new_project(project, block_schedule):
    stat = fs.FinancialStatement(name=project.name, periods=block_schedule.period)
    stat.add_factor('tax_rate', tax_rate)
    stat.add_statement(name='Environment', short_name='env')
    stat.add_statement(name='Income Statement')


    stat.env.add_account(number_of_rigs, name='Number of Miners', short_name='n_miners')
    stat.env.add_account(
        fs.arr.multiply(stat.n_miners, power_per_rig_in_joules), 
        name='Energy (J) - Miner', 
        short_name='miner_energy_in_joules',
        hide=True
    )
    stat.env.add_account(
        fs.arr.multiply(stat.n_miners, power_per_rig), 
        name='Energy - Miner', 
        short_name='miner_energy'
    )

    stat.env.add_account(total_energy(stat.miner_energy, pue), name='Energy - Infra', short_name='infra_energy')
    stat.env.add_account(fs.arr.add(stat.infra_energy, stat.miner_energy), name='Energy')

    stat.env.add_account(fs.arr.multiply(hash_rate, stat.n_miners), name='Hash Rate')
    stat.env.add_account(fs.arr.multiply(hashes, stat.n_miners), name='Hashes')




In [ ]:
env

In [ ]:
new_project(project, block_schedule)

In [ ]:
stat.env

In [ ]:
stat.istat